# BM Reports API Example

In order to use this code you will need to go onto the ELEXON Portal website [click here](https://www.elexonportal.co.uk) to register and get yourself an API key if you want to download the data yourself. The guide to using the API can be found by [clicking here](https://www.elexon.co.uk/guidance-note/bmrs-api-data-push-user-guide/). From this you should be able to work out the arguments that need to be passed to the functions below.

First need some stuff to query a website and deal with xml data types.

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import xmltodict

Use a function to build the request string to the API website and deliver the requested xml file into a variable on return.

In [2]:
def BMRS_GetXML(**kwargs):
    '''BMRS_XMLGet(api='###', report='PHYBMDATA', sd='2016-01-26', sp=3,
    bmu='T_COTPS-1',bmutype='T', leadpartyname = 'AES New Energy Limited',ngcbmuname='EAS-ASP01')'''

    url = 'https://api.bmreports.com/BMRS/{report}/v1?APIKey={api}&ServiceType=xml'.format(**kwargs)

    for key, value in kwargs.items():
        if key not in ['report','api']:
            a = "&%s=%s" % (key, value)
            url = url + a
    
    # print url to screen to check request
    print(url)
    response = requests.get(url)
    xml =  ET.fromstring(response.text)
    return xml


Using the xml file, we can easily turn this into a pandas dataframe with the following function:

In [3]:
def BMRS_Dataframe(**kwargs):
    '''Takes the sourced XML file produces a dataframe from all the children of the ITEM tag'''
    tags = []

    # get the xml file
    xml = BMRS_GetXML(**kwargs)
    
    # get tags, select first item to get tags
    root = xml.find("./responseBody/responseList/item")
    for child in root:
        tags.append(child.tag)
    
    df = pd.DataFrame(columns=tags)

    for root in xml.findall("./responseBody/responseList/item"):
        tags = []

        rowData = xmltodict.parse(ET.tostring(root))
      
        df = df.append(rowData['item'], ignore_index=True)
        
    return df

# Example: System Buy and Sell Price

Now we can build up a call to this function, including the required arguments for the particular request we want. In this case we take the system data file (called `DERSYSDATA`) that has information from the balancing market.

In [4]:
# get the report on system buy/sell prices
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='###################'
REPORT='DERSYSDATA'
FROMSETTLEMENTDATE='2020-04-01'
TOSETTLEMENTDATE='2020-04-03'
SETTLEMENTPERIOD='*'
df = BMRS_Dataframe(api=APIKEY,report=REPORT, FromSettlementDate=FROMSETTLEMENTDATE,ToSettlementDate=TOSETTLEMENTDATE,SettlementPeriod=SETTLEMENTPERIOD)

https://api.bmreports.com/BMRS/DERSYSDATA/v1?APIKey=###################&ServiceType=xml&FromSettlementDate=2020-04-01&ToSettlementDate=2020-04-03&SettlementPeriod=*


Some post processing converts the columns into appropriate data types (by default they are all `string`s), and creates a new column combining the Settlement Date and Period into a `datetime` variable.

In [5]:
# adjust the columns to appropriate types

df["settlementDate"]=pd.to_datetime(df["settlementDate"])
df["settlementPeriod"]=pd.to_numeric(df["settlementPeriod"])
df["systemSellPrice"]= df["systemSellPrice"].astype(float)
df["systemBuyPrice"]= df["systemBuyPrice"].astype(float)
df["indicativeNetImbalanceVolume"]= df["indicativeNetImbalanceVolume"].astype(float)

# make a datetime version by adding on the half hour periods

df['settlementDateTime'] = df['settlementDate'] + pd.to_timedelta( ( (df.settlementPeriod-1)/2.) , unit='h')

It looks like this:

In [6]:
df

,recordType,settlementDate,settlementPeriod,systemSellPrice,systemBuyPrice,bSADDefault,priceDerivationCode,reserveScarcityPrice,indicativeNetImbalanceVolume,sellPriceAdjustment,...,totalSystemTaggedAcceptedOfferVolume,totalSystemTaggedAcceptedBidVolume,totalSystemAdjustmentSellVolume,totalSystemAdjustmentBuyVolume,totalSystemTaggedAdjustmentSellVolume,totalSystemTaggedAdjustmentBuyVolume,activeFlag,replacementPrice,replacementPriceCalculationVolume,settlementDateTime
0,SSB,2021-09-05,1,140.00,140.00,F,P,NULL,159.1843,0.00,...,840.334,-167.195,-515.000,0.000,-515.000,0.000,Y,NaN,NaN,2021-09-05 00:00:00
1,SSB,2021-09-05,2,140.00,140.00,F,P,NULL,118.8499,0.00,...,830.487,-251.407,-515.000,53.667,-515.000,53.667,Y,NaN,NaN,2021-09-05 00:30:00
2,SSB,2021-09-05,3,140.00,140.00,F,P,NULL,78.7670,0.00,...,857.683,-186.584,-640.000,46.667,-640.000,46.667,Y,NaN,NaN,2021-09-05 01:00:00
3,SSB,2021-09-05,4,138.00,138.00,F,P,NULL,47.1244,0.00,...,798.236,-112.120,-640.000,0.000,-640.000,0.000,Y,NaN,NaN,2021-09-05 01:30:00
4,SSB,2021-09-05,5,80.00,80.00,F,N,NULL,-5.0395,0.00,...,714.884,-78.923,-640.000,0.000,-640.000,0.000,Y,NaN,NaN,2021-09-05 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,SSB,2021-09-06,25,86.00,86.00,F,N,0.00000,-77.8614,0.00,...,364.908,-457.320,0.000,15.467,0.000,15.467,Y,NaN,NaN,2021-09-06 12:00:00
73,SSB,2021-09-06,26,86.11,86.11,F,N,0.00000,-58.2733,0.00,...,351.233,-440.445,0.000,31.984,0.000,31.984,Y,NaN,NaN,2021-09-06 12:30:00
74,SSB,2021-09-06,27,86.20,86.20,F,N,NULL,-51.6417,0.00,...,346.192,-422.508,0.000,25.600,0.000,25.600,Y,NaN,NaN,2021-09-06 13:00:00
75,SSB,2021-09-06,28,86.11,86.11,F,N,NULL,-254.3025,0.00,...,174.192,-427.494,0.000,0.000,0.000,0.000,Y,NaN,NaN,2021-09-06 13:30:00


You can then output this into a csv file that can be opened in excel or imported into other programming languages.

In [7]:
# output file to csv to open in excel (or other programming language)
df.to_csv("test.csv")

# Market Prices

Another example below, reading off the market price from the report `MID`.

In [8]:
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='####################'
REPORT='MID'
FROMSETTLEMENTDATE='2020-04-01'
TOSETTLEMENTDATE='2020-04-03'
PERIOD='*'
dfMID = BMRS_Dataframe(api=APIKEY,report=REPORT, FromSettlementDate=FROMSETTLEMENTDATE,ToSettlementDate=TOSETTLEMENTDATE,Period=PERIOD)

https://api.bmreports.com/BMRS/MID/v1?APIKey=####################&ServiceType=xml&FromSettlementDate=2020-04-01&ToSettlementDate=2020-04-03&Period=*


In [9]:
# adjust the columns to appropriate types
dfMID["settlementDate"]=pd.to_datetime(dfMID["settlementDate"])
dfMID["settlementPeriod"]=pd.to_numeric(dfMID["settlementPeriod"])
dfMID["marketIndexPrice"]= dfMID["marketIndexPrice"].astype(float)
dfMID["marketIndexVolume"]= dfMID["marketIndexVolume"].astype(float)

# make a datetime version by adding on the half hour periods
dfMID['settlementDateTime'] = dfMID['settlementDate'] + pd.to_timedelta( ( (dfMID.settlementPeriod-1)/2.) , unit='h')

In [10]:
# select only APXMIDP prices
dfMID[dfMID.marketIndexDataProviderId=="APXMIDP"]

,recordType,marketIndexDataProviderId,settlementDate,settlementPeriod,marketIndexPrice,marketIndexVolume,activeFlag,settlementDateTime
0,MID,APXMIDP,2021-09-05,1,137.02,198.65,Y,2021-09-05 00:00:00
1,MID,APXMIDP,2021-09-05,2,129.49,184.50,Y,2021-09-05 00:30:00
2,MID,APXMIDP,2021-09-05,3,134.43,140.50,Y,2021-09-05 01:00:00
3,MID,APXMIDP,2021-09-05,4,133.95,203.20,Y,2021-09-05 01:30:00
4,MID,APXMIDP,2021-09-05,5,133.55,194.75,Y,2021-09-05 02:00:00
...,...,...,...,...,...,...,...,...
72,MID,APXMIDP,2021-09-06,25,187.74,861.35,Y,2021-09-06 12:00:00
73,MID,APXMIDP,2021-09-06,26,168.24,921.10,Y,2021-09-06 12:30:00
74,MID,APXMIDP,2021-09-06,27,153.45,970.90,Y,2021-09-06 13:00:00
75,MID,APXMIDP,2021-09-06,28,151.51,1037.20,Y,2021-09-06 13:30:00


# Total Generator Output and Demand

Looks like the rolling system demand records the total output on the grid at each point in time. Find it in the report "ROLSYSDEM".

In [11]:
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='####################'
REPORT='INDOITSDO'
FROMDATE='2020-04-01'
TODATE='2020-04-03'
PERIOD='*'
dfDem = BMRS_Dataframe(api=APIKEY,report=REPORT, FromDate=FROMDATE,ToDate=TODATE)

https://api.bmreports.com/BMRS/INDOITSDO/v1?APIKey=####################&ServiceType=xml&FromDate=2020-04-01&ToDate=2020-04-03


In [12]:
# select only INDO for demand
dfDem[dfDem.recordType=="INDO"]

,recordType,startTimeOfHalfHrPeriod,systemZone,settlementPeriod,publishingPeriodCommencingTime,demand,activeFlag
0,INDO,2021-09-05,N,1,2021-09-04 23:30:00,20947,Y
1,INDO,2021-09-05,N,2,2021-09-05 00:00:00,20291,Y
2,INDO,2021-09-05,N,3,2021-09-05 00:30:00,19867,Y
3,INDO,2021-09-05,N,4,2021-09-05 01:00:00,19722,Y
4,INDO,2021-09-05,N,5,2021-09-05 01:30:00,19327,Y
...,...,...,...,...,...,...,...
72,INDO,2021-09-06,N,25,2021-09-06 11:30:00,28312,Y
73,INDO,2021-09-06,N,26,2021-09-06 12:00:00,28492,Y
74,INDO,2021-09-06,N,27,2021-09-06 12:30:00,28508,Y
75,INDO,2021-09-06,N,28,2021-09-06 13:00:00,28136,Y


# Generator Output by Fuel Type

You can get all of the different outputs from each generator type in the report "FUELHH".

In [13]:
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='####################'
REPORT='FUELHH'
FROMSETTLEMENTDATE='2020-04-01'
TOSETTLEMENTDATE='2020-04-03'
PERIOD='*'
dfFuel = BMRS_Dataframe(api=APIKEY,report=REPORT, FromSettlementDate=FROMSETTLEMENTDATE,ToSettlementDate=TOSETTLEMENTDATE)


https://api.bmreports.com/BMRS/FUELHH/v1?APIKey=####################&ServiceType=xml&FromSettlementDate=2020-04-01&ToSettlementDate=2020-04-03


In [14]:
dfFuel

,recordType,startTimeOfHalfHrPeriod,settlementPeriod,ccgt,oil,coal,nuclear,wind,ps,npshyd,...,intfr,intirl,intned,intew,biomass,intnem,intelec,intifa2,intnsl,activeFlag
0,FUELHH,2021-09-05,1,8877,0,334,4523,2680,0,46,...,1608,0,806,0,1148,804,0,800,0,Y
1,FUELHH,2021-09-05,2,8630,0,287,4519,2551,0,46,...,1608,0,806,0,1149,804,0,796,0,Y
2,FUELHH,2021-09-05,3,8604,0,334,4515,2518,0,46,...,1508,0,760,0,1092,756,0,752,0,Y
3,FUELHH,2021-09-05,4,8664,0,249,4520,2485,0,46,...,1508,0,760,0,1149,756,0,750,0,Y
4,FUELHH,2021-09-05,5,8532,0,231,4539,2539,0,46,...,1508,0,762,0,1147,756,0,750,0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,FUELHH,2021-09-06,25,15620,0,1247,4395,724,384,119,...,1500,0,1032,0,1768,1000,0,992,0,Y
73,FUELHH,2021-09-06,26,15367,0,1277,4391,831,644,119,...,1500,0,1030,0,1768,1000,0,992,0,Y
74,FUELHH,2021-09-06,27,15329,0,1332,4507,994,368,132,...,1500,0,1032,0,1763,1000,0,992,0,Y
75,FUELHH,2021-09-06,28,15223,0,1405,4563,1063,94,82,...,1500,0,1032,0,1766,1000,0,992,0,Y


# Bid Offer Data -- Balancing Market

Here we get the bid offer data from a particular period. For storage units, we can look at the `T_DINO-*` entries, operated by First Hyrdo Company. There are six registered units (corresponding to six pumps at the site) that makes up GBs largest storage operator. Each unit can supply/demand up to 600MW.

In [17]:
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='####################'
REPORT='BOD'
SETTLEMENTDATE='2020-04-01'
SETTLEMENTPERIOD='32'
BMUNITID='*'
BMUNITTYPE='*'
LEADPARTYNAME='*'
NGCBMUNITNAME='*'
dfBOD = BMRS_Dataframe(api=APIKEY,report=REPORT,SettlementDate=FROMSETTLEMENTDATE,SettlementPeriod=SETTLEMENTPERIOD,BMUnitId=BMUNITID,LeadPartyName=LEADPARTYNAME,NGCBMUnitName=NGCBMUNITNAME)


https://api.bmreports.com/BMRS/BOD/v1?APIKey=####################&ServiceType=xml&SettlementDate=2020-04-01&SettlementPeriod=32&BMUnitId=*&LeadPartyName=*&NGCBMUnitName=*


In [22]:
dfBOD[dfBOD.bmUnitID=="T_DINO-1"]

,recordType,bmUnitID,bMUnitType,leadPartyName,ngcBMUnitName,settlementDate,settlementPeriod,bidOfferPairNumber,timeFrom,bidOfferLevelFrom,timeTo,bidOfferLevelTo,bidPrice,offerPrice,activeFlag
2002,BOD,T_DINO-1,T,First Hydro Company,DINO-1,2021-09-06,33,1,2021-09-06 15:00:00,587,2021-09-06 15:30:00,587,-150.00,2000.00,Y
2003,BOD,T_DINO-1,T,First Hydro Company,DINO-1,2021-09-06,33,-1,2021-09-06 15:00:00,-587,2021-09-06 15:30:00,-587,-150.00,2000.00,Y


Here we see that the bid is extremely low, -£150, meaning that they will need to be paid to buy electricity. The sale price is also very high, 2000, meaning they may need to keep this pump in reserve (perhaps receiving payments to be on standby). Another one of the pumps has a much lower spread making it more likely to be used, perhaps buying electricity at this time to sell back in a couple of hours at peak time.

In [23]:
dfBOD[dfBOD.bmUnitID=="T_DINO-6"]

,recordType,bmUnitID,bMUnitType,leadPartyName,ngcBMUnitName,settlementDate,settlementPeriod,bidOfferPairNumber,timeFrom,bidOfferLevelFrom,timeTo,bidOfferLevelTo,bidPrice,offerPrice,activeFlag
2012,BOD,T_DINO-6,T,First Hydro Company,DINO-6,2021-09-06,33,1,2021-09-06 15:00:00,587,2021-09-06 15:30:00,587,60.00,1100.00,Y
2013,BOD,T_DINO-6,T,First Hydro Company,DINO-6,2021-09-06,33,-1,2021-09-06 15:00:00,-587,2021-09-06 15:30:00,-587,60.00,1100.00,Y
